Selection Impact (T-tests)

In [1]:
import os
import pandas as pd
from scipy.stats import ttest_rel
import numpy as np
def t_test(original, selection):
    """Comparing method"""
    def two_tailed_t_test(original, selection):
        n_d = len(selection)
        n_c = len(original)
        n = min(n_d, n_c)
        t, p = ttest_rel(original[:n], selection[:n])
        if np.isnan(t):
            t, p = 0, 1
        return {"t-stats":t, "p-value":p}

    def one_tailed_t_test(original, selection, direction):
        two_tail = two_tailed_t_test(original, selection)
        t, p_two = two_tail['t-stats'], two_tail['p-value']
        if direction == 'positive':
            if t > 0 :
                p = p_two * 0.5
            else:
                p = 1 - p_two * 0.5
        else:
            if t < 0:
                p = p_two * 0.5
            else:
                p = 1 - p_two * 0.5
        return {"t-stats":t, "p-value":p}

    result = {}
    result['two_tail'] = two_tailed_t_test(original, selection)
    result['one_tail_pos'] = one_tailed_t_test(original, selection, 'positive')
    result['one_tail_neg'] = one_tailed_t_test(original, selection, 'negative')
    return result


def evaluate_score(original, selection):
    alpha =  0.05
    results = t_test(original, selection)
    difference = 'insignificant'

    if results['two_tail']['p-value'] < alpha:
        if results['one_tail_neg']['p-value'] < alpha:
            difference = 'positive'
        if results['one_tail_pos']['p-value'] < alpha:
            difference = 'negative'

    return difference


T-tests 

ARS dataset

In [5]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../../results/ars-selection/CRAIGPB', '../../results/ars-selection/GLISTERPB', '../../results/ars-selection/GradMatchPB', '../../results/ars-selection/Random']
ratio_path = ['/ars_0.05', '/ars_0.1', '/ars_0.2', '/ars_0.3', '/ars_0.5']
models = ['Logreg', 'MLP', 'SVM']

directory_full = '../../results/ars-selection/Full/ars_1'
# Metrics tables
index_labels = ['negative-f', 'insignificant-f','positive-f','negative-u', 'insignificant-u','positive-u','negative-c', 'insignificant-c','positive-c', 'insignificant-f-u and positive-c' ]
# Initialize influence table
cols = ['ratio', 'system', 'time', 'acc', 'F1_score', 'Precision', 'Recall', 'SPD_gender', 'EOD_gender', 'AOD_gender', 'DI_gender', 'DcI_gender']
compute_influence = pd.DataFrame(columns=cols)


# Initialize full result and cost DataFrames
df_full_result = pd.DataFrame()
df_full_cost = pd.DataFrame()
df_full_cost_time = pd.DataFrame()

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result = pd.DataFrame()
    df_full_cost = pd.DataFrame()
    df_full_cost_time = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'SVM':
               df_r = pd.read_csv(file_path).iloc[34:69, 1:]
           else:
               df_r = pd.read_csv(file_path).iloc[54:69, 1:]
           df_full_result = pd.concat([df_full_result, df_r.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'SVM':
                df_ct = pd.read_csv(file_path).iloc[34:69, 1:]  
            else:
                df_ct = pd.read_csv(file_path).iloc[54:69, 1:]  
            df_full_cost_time = pd.concat([df_full_cost_time, df_ct.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'SVM':
                df_c = pd.read_csv(file_path).iloc[34:69, 1:]
            else:
                df_c = pd.read_csv(file_path).iloc[54:69, 1:]
            df_full_cost = pd.concat([df_full_cost, df_c.abs()], ignore_index=True)

    print('**********************')
    print(model)


    for col in ['DI_gender']:
        df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg = df_full_result.mean()
    cost_avg = df_full_cost.mean()
    cost_time_avg = df_full_cost_time.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{col + '_avg': [result_avg[col]] for col in result_avg.index},
        **{col + '_avg': [cost_avg[col]] for col in cost_avg.index},
        **{col + '_avg': [cost_time_avg[col]] for col in cost_time_avg.index},
        'dataset': ['ars'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection = pd.DataFrame()
           df_full_cost_selection = pd.DataFrame()
           df_full_cost_time_selection = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_f = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_f = pd.read_csv(file_path).iloc[34:49, 1:].abs()
                    df_full_result_selection = pd.concat([df_full_result_selection, df_to_add_f], ignore_index=True)
                    
                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_t = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_t = pd.read_csv(file_path).iloc[34:49, 1:].abs()

                    df_full_cost_time_selection = pd.concat([df_full_cost_time_selection, df_to_add_t], ignore_index=True)
        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'SVM':
                        df_to_add_c = pd.read_csv(file_path).iloc[14:49, 1:].abs()
                    else:
                        df_to_add_c = pd.read_csv(file_path).iloc[34:49, 1:].abs()
                    df_full_cost_selection = pd.concat([df_full_cost_selection, df_to_add_c], ignore_index=True)
           
                
           for col in ['DI_gender']:
             df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg = df_full_result_selection.mean()
           cost_sel_avg = df_full_cost_selection.mean()
           cost_time_sel_avg = df_full_cost_time_selection.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection['Full_training_time'].to_numpy(), df_full_cost_time['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost['accuracy'].to_numpy(), df_full_cost_selection['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result['F1_score'].to_numpy(), df_full_result_selection['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result['Precision'].to_numpy(), df_full_result_selection['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result['Recall'].to_numpy(), df_full_result_selection['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection['SPD_gender'].to_numpy(), df_full_result['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection['EOD_gender'].to_numpy(), df_full_result['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection['AOD_gender'].to_numpy(), df_full_result['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection['DI_gender'].to_numpy(), df_full_result['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection['DcI_gender'].to_numpy(), df_full_result['DcI_gender'].to_numpy())



           avg_sel_data = pd.DataFrame({
            **{col + '_avg': [result_sel_avg[col]] for col in result_sel_avg.index},
            **{col + '_avg': [cost_sel_avg[col]] for col in cost_sel_avg.index},
            **{col + '_avg': [cost_time_sel_avg[col]] for col in cost_time_sel_avg.index},
           'dataset': ['ars'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : ' ',
           'test_EOD_age' : ' ',
           'test_AOD_age' : ' ', 
           'test_DI_age' : ' ', 
           'test_DcI_age' : ' ', 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ', 
           'test_DcI_race' : ' ', 
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../../results/test/ars_ttest_results_epochs_intervals.csv', index=False)



**********************
Logreg


/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: Ru

**********************
MLP


/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: Ru

**********************
SVM


/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/zeyang/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: Ru

Adult dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/adult-selection/CRAIGPB', '../results/adult-selection/GLISTERPB', '../results/adult-selection/GradMatchPB', '../results/adult-selection/Random']
ratio_path = ['/adult_0.05', '/adult_0.1', '/adult_0.2','/adult_0.3', '/adult_0.5']
models = ['Logreg', 'MLP', 'SVM']  

directory_full = '../results/adult-selection/Full/adult_1'


# Initialize full result and cost DataFrames
df_full_result = pd.DataFrame()
df_full_cost = pd.DataFrame()
df_full_cost_time = pd.DataFrame()

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result = pd.DataFrame()
    df_full_cost = pd.DataFrame()
    df_full_cost_time = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
             df_r = pd.read_csv(file_path).iloc[89:199, 1:]
           else :
             df_r = pd.read_csv(file_path).iloc[54:99, 1:]
           df_full_result = pd.concat([df_full_result, df_r.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
             df_ct = pd.read_csv(file_path).iloc[89:199, 1:]
            else :
             df_ct = pd.read_csv(file_path).iloc[54:99, 1:]
            df_full_cost_time = pd.concat([df_full_cost_time, df_ct.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
             df_c = pd.read_csv(file_path).iloc[89:199, 1:]
            else :
             df_c = pd.read_csv(file_path).iloc[54:99, 1:]
            df_full_cost = pd.concat([df_full_cost, df_c.abs()], ignore_index=True)

    print('**********************')
    print(model)

    for col in ['DI_gender', 'DI_age', 'DI_race']:
        df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg = df_full_result.mean()
    cost_avg = df_full_cost.mean()
    cost_time_avg = df_full_cost_time.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{col + '_avg': [result_avg[col]] for col in result_avg.index},
        **{col + '_avg': [cost_avg[col]] for col in cost_avg.index},
        **{col + '_avg': [cost_time_avg[col]] for col in cost_time_avg.index},
        'dataset': ['adult'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection = pd.DataFrame()
           df_full_cost_selection = pd.DataFrame()
           df_full_cost_time_selection = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_f = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                       
                    else:
                       df_to_add_f = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    df_full_result_selection = pd.concat([df_full_result_selection, df_to_add_f], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_t = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                    else:
                       df_to_add_t = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    df_full_cost_time_selection = pd.concat([df_full_cost_time_selection, df_to_add_t], ignore_index=True)
    

        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_c = pd.read_csv(file_path).iloc[69:179, 1:].abs()
                    else:
                       df_to_add_c = pd.read_csv(file_path).iloc[34:79, 1:].abs()
                    df_full_cost_selection = pd.concat([df_full_cost_selection, df_to_add_c], ignore_index=True)
           
                
           for col in ['DI_gender', 'DI_age', 'DI_race']:
             df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg = df_full_result_selection.mean()
           cost_sel_avg = df_full_cost_selection.mean()
           cost_time_sel_avg = df_full_cost_time_selection.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection['Full_training_time'].to_numpy(), df_full_cost_time['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost['accuracy'].to_numpy(), df_full_cost_selection['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result['F1_score'].to_numpy(), df_full_result_selection['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result['Precision'].to_numpy(), df_full_result_selection['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result['Recall'].to_numpy(), df_full_result_selection['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection['SPD_gender'].to_numpy(), df_full_result['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection['EOD_gender'].to_numpy(), df_full_result['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection['AOD_gender'].to_numpy(), df_full_result['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection['DI_gender'].to_numpy(), df_full_result['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection['DcI_gender'].to_numpy(), df_full_result['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection['SPD_age'].to_numpy(), df_full_result['SPD_age'].to_numpy())
           # EOD gender
           test_eod_a = evaluate_score(df_full_result_selection['EOD_age'].to_numpy(), df_full_result['EOD_age'].to_numpy())
           # AOD gender
           test_aod_a = evaluate_score(df_full_result_selection['AOD_age'].to_numpy(), df_full_result['AOD_age'].to_numpy())
           # DI gender
           test_di_a = evaluate_score(df_full_result_selection['DI_age'].to_numpy(), df_full_result['DI_age'].to_numpy())
           # DcI gender
           test_dci_a = evaluate_score(df_full_result_selection['DcI_age'].to_numpy(), df_full_result['DcI_age'].to_numpy())

           # SPD race
           test_spd_r = evaluate_score(df_full_result_selection['SPD_race'].to_numpy(), df_full_result['SPD_race'].to_numpy())
           # EOD race
           test_eod_r = evaluate_score(df_full_result_selection['EOD_race'].to_numpy(), df_full_result['EOD_race'].to_numpy())
           # AOD race
           test_aod_r = evaluate_score(df_full_result_selection['AOD_race'].to_numpy(), df_full_result['AOD_race'].to_numpy())
           # DI race
           test_di_r = evaluate_score(df_full_result_selection['DI_race'].to_numpy(), df_full_result['DI_race'].to_numpy())
           # DcI race
           test_dci_r = evaluate_score(df_full_result_selection['DcI_race'].to_numpy(), df_full_result['DcI_race'].to_numpy())



           avg_sel_data = pd.DataFrame({
            **{col + '_avg': [result_sel_avg[col]] for col in result_sel_avg.index},
            **{col + '_avg': [cost_sel_avg[col]] for col in cost_sel_avg.index},
            **{col + '_avg': [cost_time_sel_avg[col]] for col in cost_time_sel_avg.index},
           'dataset': ['adult'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : [test_spd_r],
           'test_EOD_race' : [test_eod_r],
           'test_AOD_race' : [test_aod_r], 
           'test_DI_race' : [test_di_r],
           'test_DcI_race' : [test_dci_r],
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/adult_ttest_results_epochs_intervals.csv', index=False)



KDD dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/kdd-selection/CRAIGPB', '../results/kdd-selection/GLISTERPB', '../results/kdd-selection/GradMatchPB', '../results/kdd-selection/Random']
ratio_path = ['/kdd_0.05', '/kdd_0.1', '/kdd_0.2','/kdd_0.3', '/kdd_0.5']
models = ['Logreg', 'MLP', 'SVM']  

directory_full = '../results/kdd-selection/Full/kdd_1'


# Initialize full result and cost DataFrames
df_full_result = pd.DataFrame()
df_full_cost = pd.DataFrame()
df_full_cost_time = pd.DataFrame()

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result = pd.DataFrame()
    df_full_cost = pd.DataFrame()
    df_full_cost_time = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
              df_r = pd.read_csv(file_path).iloc[34:79, 1:]
           else:
              df_r = pd.read_csv(file_path).iloc[24:79, 1:]
           df_full_result = pd.concat([df_full_result, df_r.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_ct = pd.read_csv(file_path).iloc[34:79, 1:]
            else:
               df_ct = pd.read_csv(file_path).iloc[24:79, 1:]
            df_full_cost_time = pd.concat([df_full_cost_time, df_ct.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_c = pd.read_csv(file_path).iloc[34:79, 1:]
            else:
               df_c = pd.read_csv(file_path).iloc[24:79, 1:]
            df_full_cost = pd.concat([df_full_cost, df_c.abs()], ignore_index=True)

    print('**********************')
    print(model)

    for col in ['DI_gender', 'DI_age', 'DI_race']:
      df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg = df_full_result.mean()
    cost_avg = df_full_cost.mean()
    cost_time_avg = df_full_cost_time.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{col + '_avg': [result_avg[col]] for col in result_avg.index},
        **{col + '_avg': [cost_avg[col]] for col in cost_avg.index},
        **{col + '_avg': [cost_time_avg[col]] for col in cost_time_avg.index},
        'dataset': ['kdd'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection = pd.DataFrame()
           df_full_cost_selection = pd.DataFrame()
           df_full_cost_time_selection = pd.DataFrame()
        
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_f = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_f = pd.read_csv(file_path).iloc[4:59, 1:].abs()
                    df_full_result_selection = pd.concat([df_full_result_selection, df_to_add_f], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_t = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_t = pd.read_csv(file_path).iloc[4:59, 1:].abs()
                    df_full_cost_time_selection = pd.concat([df_full_cost_time_selection, df_to_add_t], ignore_index=True)
    

        
                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP':
                       df_to_add_c = pd.read_csv(file_path).iloc[14:59, 1:].abs()
                    else:
                       df_to_add_c = pd.read_csv(file_path).iloc[4:59, 1:].abs()
                    df_full_cost_selection = pd.concat([df_full_cost_selection, df_to_add_c], ignore_index=True)
           
                
           for col in ['DI_gender', 'DI_age', 'DI_race']:
             df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))

           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg = df_full_result_selection.mean()
           cost_sel_avg = df_full_cost_selection.mean()
           cost_time_sel_avg = df_full_cost_time_selection.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection['Full_training_time'].to_numpy(), df_full_cost_time['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost['accuracy'].to_numpy(), df_full_cost_selection['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result['F1_score'].to_numpy(), df_full_result_selection['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result['Precision'].to_numpy(), df_full_result_selection['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result['Recall'].to_numpy(), df_full_result_selection['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection['SPD_gender'].to_numpy(), df_full_result['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection['EOD_gender'].to_numpy(), df_full_result['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection['AOD_gender'].to_numpy(), df_full_result['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection['DI_gender'].to_numpy(), df_full_result['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection['DcI_gender'].to_numpy(), df_full_result['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection['SPD_age'].to_numpy(), df_full_result['SPD_age'].to_numpy())
           # EOD gender
           test_eod_a = evaluate_score(df_full_result_selection['EOD_age'].to_numpy(), df_full_result['EOD_age'].to_numpy())
           # AOD gender
           test_aod_a = evaluate_score(df_full_result_selection['AOD_age'].to_numpy(), df_full_result['AOD_age'].to_numpy())
           # DI gender
           test_di_a = evaluate_score(df_full_result_selection['DI_age'].to_numpy(), df_full_result['DI_age'].to_numpy())
           # DcI gender
           test_dci_a = evaluate_score(df_full_result_selection['DcI_age'].to_numpy(), df_full_result['DcI_age'].to_numpy())

           # SPD race
           test_spd_r = evaluate_score(df_full_result_selection['SPD_race'].to_numpy(), df_full_result['SPD_race'].to_numpy())
           # EOD race
           test_eod_r = evaluate_score(df_full_result_selection['EOD_race'].to_numpy(), df_full_result['EOD_race'].to_numpy())
           # AOD race
           test_aod_r = evaluate_score(df_full_result_selection['AOD_race'].to_numpy(), df_full_result['AOD_race'].to_numpy())
           # DI race
           test_di_r = evaluate_score(df_full_result_selection['DI_race'].to_numpy(), df_full_result['DI_race'].to_numpy())
           # DcI race
           test_dci_r = evaluate_score(df_full_result_selection['DcI_race'].to_numpy(), df_full_result['DcI_race'].to_numpy())



           avg_sel_data = pd.DataFrame({
            **{col + '_avg': [result_sel_avg[col]] for col in result_sel_avg.index},
            **{col + '_avg': [cost_sel_avg[col]] for col in cost_sel_avg.index},
            **{col + '_avg': [cost_time_sel_avg[col]] for col in cost_time_sel_avg.index},
           'dataset': ['kdd'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : [test_spd_r],
           'test_EOD_race' : [test_eod_r],
           'test_AOD_race' : [test_aod_r], 
           'test_DI_race' : [test_di_r],
           'test_DcI_race' : [test_dci_r],
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/kdd_ttest_results_epochs_intervals.csv', index=False)



DC dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/dc-selection/CRAIGPB', '../results/dc-selection/GLISTERPB', '../results/dc-selection/GradMatchPB', '../results/dc-selection/Random']
ratio_path = ['/dc_0.05', '/dc_0.1', '/dc_0.2','/dc_0.3', '/dc_0.5']
models = ['LogReg', 'MLP', 'SVM']  

directory_full = '../results/dc-selection/Full/dc_1'


# Initialize full result and cost DataFrames
df_full_result = pd.DataFrame()
df_full_cost = pd.DataFrame()
df_full_cost_time = pd.DataFrame()

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result = pd.DataFrame()
    df_full_cost = pd.DataFrame()
    df_full_cost_time = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "fair_metrics_"+ model in filename :
           file_path = os.path.join(directory_full, filename)
           if model == 'MLP':
               df_r = pd.read_csv(file_path).iloc[44:99, 1:]
           elif model == 'SVM':
               df_r = pd.read_csv(file_path).iloc[29:99, 1:]
           else:
               df_r = pd.read_csv(file_path).iloc[124:269, 1:]
           df_full_result = pd.concat([df_full_result, df_r.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_ct = pd.read_csv(file_path).iloc[44:99, 1:]
            elif model == 'SVM':
               df_ct = pd.read_csv(file_path).iloc[29:99, 1:]
            else:
               df_ct = pd.read_csv(file_path).iloc[124:269, 1:]
            df_full_cost_time = pd.concat([df_full_cost_time, df_ct.abs()], ignore_index=True)

        if "cost_metrics_" + model in filename :
            file_path = os.path.join(directory_full, filename)
            if model == 'MLP':
               df_c = pd.read_csv(file_path).iloc[44:99, 1:]
            elif model == 'SVM':
               df_c = pd.read_csv(file_path).iloc[29:99, 1:]
            else:
               df_c = pd.read_csv(file_path).iloc[124:269, 1:]
            df_full_cost = pd.concat([df_full_cost, df_c.abs()], ignore_index=True)

    print('**********************')
    print(model)

    for col in ['DI_gender', 'DI_age']:
      df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))
    

    ### Calcul de valeurs moyenne pour le full 
    result_avg = df_full_result.mean()
    cost_avg = df_full_cost.mean()
    cost_time_avg = df_full_cost_time.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{col + '_avg': [result_avg[col]] for col in result_avg.index},
        **{col + '_avg': [cost_avg[col]] for col in cost_avg.index},
        **{col + '_avg': [cost_time_avg[col]] for col in cost_time_avg.index},
        'dataset': ['dc'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection = pd.DataFrame()
           df_full_cost_selection = pd.DataFrame()
           df_full_cost_time_selection = pd.DataFrame()
        
           
           # Cost file processing
           for filename in os.listdir(directory):
                if "fair_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_f = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_f = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else:
                        df_to_add_f = pd.read_csv(file_path).iloc[124:269, 1:].abs()
                    df_full_result_selection = pd.concat([df_full_result_selection, df_to_add_f], ignore_index=True)
                    

                if (("cost_metrics_" + model in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_t = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_t = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else: 
                        df_to_add_t = pd.read_csv(file_path).iloc[124:269, 1:].abs()
                    df_full_cost_time_selection = pd.concat([df_full_cost_time_selection, df_to_add_t], ignore_index=True)
    

                

                if "cost_metrics_" + model in filename:
                    file_path = os.path.join(directory, filename)
                    if model == 'MLP': 
                        df_to_add_c = pd.read_csv(file_path).iloc[44:99, 1:].abs()
                    elif model == 'SVM':
                        df_to_add_c = pd.read_csv(file_path).iloc[29:99, 1:].abs()
                    else:
                        df_to_add_c = pd.read_csv(file_path).iloc[124:269, 1:].abs()
                    df_full_cost_selection = pd.concat([df_full_cost_selection, df_to_add_c], ignore_index=True)
           
                
           for col in ['DI_gender', 'DI_age']:
             df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
           
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg = df_full_result_selection.mean()
           cost_sel_avg = df_full_cost_selection.mean()
           cost_time_sel_avg = df_full_cost_time_selection.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection['Full_training_time'].to_numpy(), df_full_cost_time['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost['accuracy'].to_numpy(), df_full_cost_selection['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result['F1_score'].to_numpy(), df_full_result_selection['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result['Precision'].to_numpy(), df_full_result_selection['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result['Recall'].to_numpy(), df_full_result_selection['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection['SPD_gender'].to_numpy(), df_full_result['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection['EOD_gender'].to_numpy(), df_full_result['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection['AOD_gender'].to_numpy(), df_full_result['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection['DI_gender'].to_numpy(), df_full_result['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection['DcI_gender'].to_numpy(), df_full_result['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection['SPD_age'].to_numpy(), df_full_result['SPD_age'].to_numpy())
           # EOD age
           test_eod_a = evaluate_score(df_full_result_selection['EOD_age'].to_numpy(), df_full_result['EOD_age'].to_numpy())
           # AOD age
           test_aod_a = evaluate_score(df_full_result_selection['AOD_age'].to_numpy(), df_full_result['AOD_age'].to_numpy())
           # DI age
           test_di_a = evaluate_score(df_full_result_selection['DI_age'].to_numpy(), df_full_result['DI_age'].to_numpy())
           # DcI age
           test_dci_a = evaluate_score(df_full_result_selection['DcI_age'].to_numpy(), df_full_result['DcI_age'].to_numpy())

        



           avg_sel_data = pd.DataFrame({
            **{col + '_avg': [result_sel_avg[col]] for col in result_sel_avg.index},
            **{col + '_avg': [cost_sel_avg[col]] for col in cost_sel_avg.index},
            **{col + '_avg': [cost_time_sel_avg[col]] for col in cost_time_sel_avg.index},
           'dataset': ['dc'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ',
           'test_DcI_race' : ' ',
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/dc_ttest_results_epochs_intervals.csv', index=False)



MobiAct dataset

In [ ]:
import os
import pandas as pd
import numpy as np
# Define paths and initialize variables
systems_path = ['../results/mobiact-selection/CRAIGPB', '../results/mobiact-selection/GLISTERPB', '../results/mobiact-selection/GradMatchPB', '../results/mobiact-selection/Random']
ratio_path = ['/mobiact_0.05', '/mobiact_0.1', '/mobiact_0.2','/mobiact_0.3', '/mobiact_0.5']
models = ['MLP']  

directory_full = '../results/mobiact-selection/Full/mobiact_1'


# Initialize full result and cost DataFrames
df_full_result = pd.DataFrame()
df_full_cost = pd.DataFrame()
df_full_cost_time = pd.DataFrame()

i = 0
files = []

df_average_results = pd.DataFrame()

# Load full system results for comparisons
for model in models: 

    df_full_result = pd.DataFrame()
    df_full_cost = pd.DataFrame()
    df_full_cost_time = pd.DataFrame()

    for filename in os.listdir(directory_full):
        if "train_mobiact_fair_metrics_" in filename :
           file_path = os.path.join(directory_full, filename)
           df_r = pd.read_csv(file_path).iloc[29:79, 1:]
           df_full_result = pd.concat([df_full_result, df_r.abs()], ignore_index=True)

        if "train_mobiact_cost_metrics_"  in filename and filename.endswith("_0.csv"):
            file_path = os.path.join(directory_full, filename)
            df_ct = pd.read_csv(file_path).iloc[29:79, 1:]
            df_full_cost_time = pd.concat([df_full_cost_time, df_ct.abs()], ignore_index=True)

        if "train_mobiact_cost_metrics_"  in filename :
            file_path = os.path.join(directory_full, filename)
            df_c = pd.read_csv(file_path).iloc[29:79, 1:]
            df_full_cost = pd.concat([df_full_cost, df_c.abs()], ignore_index=True)

    print('**********************')
    print(model)

    for col in ['DI_gender', 'DI_age']:
      df_full_result[col] = df_full_result[col].apply(lambda x: abs(1 - x))

    ### Calcul de valeurs moyenne pour le full 
    result_avg = df_full_result.mean()
    cost_avg = df_full_cost.mean()
    cost_time_avg = df_full_cost_time.mean()

    # Ligne de valeurs moyennes de full 
    avg_data = pd.DataFrame({
        **{col + '_avg': [result_avg[col]] for col in result_avg.index},
        **{col + '_avg': [cost_avg[col]] for col in cost_avg.index},
        **{col + '_avg': [cost_time_avg[col]] for col in cost_time_avg.index},
        'dataset': ['mobiact'],
        'model': [model],
        'system': ['Full'],
        'ratio': [1],
    })

    df_average_results = pd.concat([df_average_results, avg_data], ignore_index=True)

    # Lire les fichiers de système de selection par système et par ratio

    for directory_path_1 in systems_path:
       for directory_path_2 in ratio_path:
           directory = directory_path_1 + directory_path_2
           system = directory_path_1.split('/')[-1]           
        
           df_full_result_selection = pd.DataFrame()
           df_full_cost_selection = pd.DataFrame()
           df_full_cost_time_selection = pd.DataFrame()
        
           
           # Cost file processing
           for filename in os.listdir(directory):
                if "train_mobiact_fair_metrics_"  in filename:
                    file_path = os.path.join(directory, filename)
                    df_to_add_f = pd.read_csv(file_path).iloc[29:79, 1:].abs()
                    df_full_result_selection = pd.concat([df_full_result_selection, df_to_add_f], ignore_index=True)
                    

                if (("train_mobiact_cost_metrics_"  in filename) and (filename.endswith("_0.csv"))):
                    file_path = os.path.join(directory, filename)
                    df_to_add_t = pd.read_csv(file_path).iloc[29:79, 1:].abs()
                    df_full_cost_time_selection = pd.concat([df_full_cost_time_selection, df_to_add_t], ignore_index=True)
    

                

                if "train_mobiact_cost_metrics_"  in filename:
                    file_path = os.path.join(directory, filename)
                    df_to_add_c = pd.read_csv(file_path).iloc[29:79, 1:].abs()
                    df_full_cost_selection = pd.concat([df_full_cost_selection, df_to_add_c], ignore_index=True)
           
                
           for col in ['DI_gender', 'DI_age']:
             df_full_result_selection[col] = df_full_result_selection[col].apply(lambda x: abs(1 - x))
    
           ### Calcul de valeurs moyenne pour le full 
           result_sel_avg = df_full_result_selection.mean()
           cost_sel_avg = df_full_cost_selection.mean()
           cost_time_sel_avg = df_full_cost_time_selection.mean()

           ## Faire les t-test
           # Cost
           # time
           test_time = evaluate_score(df_full_cost_time_selection['Full_training_time'].to_numpy(), df_full_cost_time['Full_training_time'].to_numpy()) 
           
           # Model quality
           # accuracy
           test_acc = evaluate_score(df_full_cost['accuracy'].to_numpy(), df_full_cost_selection['accuracy'].to_numpy()) 
           # F1-score
           test_f1 = evaluate_score(df_full_result['F1_score'].to_numpy(), df_full_result_selection['F1_score'].to_numpy()) 
           #precision
           test_precision = evaluate_score(df_full_result['Precision'].to_numpy(), df_full_result_selection['Precision'].to_numpy()) 
           # recall 
           test_recall = evaluate_score(df_full_result['Recall'].to_numpy(), df_full_result_selection['Recall'].to_numpy()) 

           # Fairness
           # SPD gender
           test_spd_g = evaluate_score(df_full_result_selection['SPD_gender'].to_numpy(), df_full_result['SPD_gender'].to_numpy())
           # EOD gender
           test_eod_g = evaluate_score(df_full_result_selection['EOD_gender'].to_numpy(), df_full_result['EOD_gender'].to_numpy())
           # AOD gender
           test_aod_g = evaluate_score(df_full_result_selection['AOD_gender'].to_numpy(), df_full_result['AOD_gender'].to_numpy())
           # DI gender
           test_di_g = evaluate_score(df_full_result_selection['DI_gender'].to_numpy(), df_full_result['DI_gender'].to_numpy())
           # DcI gender
           test_dci_g = evaluate_score(df_full_result_selection['DcI_gender'].to_numpy(), df_full_result['DcI_gender'].to_numpy())

           # SPD age
           test_spd_a = evaluate_score(df_full_result_selection['SPD_age'].to_numpy(), df_full_result['SPD_age'].to_numpy())
           # EOD age
           test_eod_a = evaluate_score(df_full_result_selection['EOD_age'].to_numpy(), df_full_result['EOD_age'].to_numpy())
           # AOD age
           test_aod_a = evaluate_score(df_full_result_selection['AOD_age'].to_numpy(), df_full_result['AOD_age'].to_numpy())
           # DI age
           test_di_a = evaluate_score(df_full_result_selection['DI_age'].to_numpy(), df_full_result['DI_age'].to_numpy())
           # DcI age
           test_dci_a = evaluate_score(df_full_result_selection['DcI_age'].to_numpy(), df_full_result['DcI_age'].to_numpy())

        



           avg_sel_data = pd.DataFrame({
            **{col + '_avg': [result_sel_avg[col]] for col in result_sel_avg.index},
            **{col + '_avg': [cost_sel_avg[col]] for col in cost_sel_avg.index},
            **{col + '_avg': [cost_time_sel_avg[col]] for col in cost_time_sel_avg.index},
           'dataset': ['mobiact'],
           'model': [model],
           'system': [system],
           'ratio': [directory_path_2.split('_')[-1]],
           'test_time' : [test_time], 
           'test_acc' : [test_acc], 
           'test_f1' : [test_f1], 
           'test_precision' : [test_precision], 
           'test_recall' : [test_recall],
           'test_SPD_gender' : [test_spd_g], 
           'test_EOD_gender' : [test_eod_g], 
           'test_AOD_gender' : [test_aod_g], 
           'test_DI_gender' : [test_di_g], 
           'test_DcI_gender' : [test_dci_g], 

           'test_SPD_age' : [test_spd_a],
           'test_EOD_age' : [test_eod_a],
           'test_AOD_age' : [test_aod_a], 
           'test_DI_age' : [test_di_a], 
           'test_DcI_age' : [test_dci_a], 

           'test_SPD_race' : ' ',
           'test_EOD_race' : ' ',
           'test_AOD_race' : ' ', 
           'test_DI_race' : ' ',
           'test_DcI_race' : ' ',
            })
           
           df_average_results = pd.concat([df_average_results, avg_sel_data], ignore_index=True)




 

df_average_results.to_csv('../results/test/mobiact_ttest_epochs_intervals.csv', index=False)



Post-processing T-test results 

Filtering T-test results based on model utility and fairness constraints

Focus on cases where model accuracy is severely degraded, in such cases, fairness impact is considered insignificant.

In [ ]:
import pandas as pd


def false_positives(path= '../results/test/ars_ttest_results_epochs_intervals.csv', x = 10):
    # Initialize the reference accuracy
    ref_accuracy = None
    # DataFrame to hold rows that are 'false better'
    false_better_df = pd.DataFrame()

    df = pd.read_csv(path) 

    # Columns to check for the value 'positive'
    columns_to_modify = ['test_SPD_gender', 'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
                        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
                        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race']

    for index, row in df.iterrows():
        if row['system'] == 'Full':
            ref_accuracy = row['accuracy_avg']  # Set the reference accuracy
        elif ref_accuracy is not None:  # Ensure there is a reference to compare against
            current_accuracy = row['accuracy_avg']
            if current_accuracy <= ref_accuracy - x and row['system'] != 'Random' and row['ratio'] != 0.5:
                # Add row to DataFrame using pd.concat
                false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)
                # Modify specific columns if their values are 'positive'
                for col in columns_to_modify:
                    if row[col] == 'positive':
                        df.at[index, col] = 'insignificant-p'


    # Save 'false better' rows to a new CSV file
    path_o= '../results/test/false_better_' + str(x) + '_' + path.split('/')[1]
    false_better_df.to_csv(path_o, index=False)


    # Save the modified DataFrame to a new CSV file
    path_m= '../results/test/positive_insig_'+ str(x) + '_' + path.split('/')[1]
    df.to_csv(path_m, index=False)

    return len(false_better_df)


In [ ]:
k = 5
ars = false_positives('../results/test/ars_ttest_results_epochs_intervals.csv', k)
dc = false_positives('../results/test/dc_ttest_results_epochs_intervals.csv', k)
kdd = false_positives('../results/test/kdd_ttest_results_epochs_intervals.csv', k)
adult = false_positives('../results/test/adult_ttest_results_epochs_intervals.csv', k)
mobiact = false_positives('../results/test/mobiact_ttest_epochs_intervals.csv', k)

Fairness constraint: the impact is considered significant (positive or negative) if the difference between the full metric and the selection metric exceeds 1%, or 0.01 for DI.

In [ ]:
import pandas as pd

def false_positives_fairness(path=None, thr_di=1, thr_other=0.01, output_false_positives=None, output=None):
    # Load the CSV file
    df = pd.read_csv(path)

    # Initialize DataFrames
    false_better_df = pd.DataFrame()

    # Columns to check for the value 'positive'
    columns_to_modify = [
        'test_SPD_gender', 'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race'
    ]

    # Columns with the average metrics
    columns_to_check = [
        'SPD_gender_avg', 'EOD_gender_avg', 'AOD_gender_avg', 'DI_gender_avg', 'DcI_gender_avg',
        'SPD_age_avg', 'EOD_age_avg', 'AOD_age_avg', 'DI_age_avg', 'DcI_age_avg',
        'SPD_race_avg', 'EOD_race_avg', 'AOD_race_avg', 'DI_race_avg', 'DcI_race_avg'
    ]

    # Initialize references dictionary
    references = {col: None for col in columns_to_check if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'positive':
                    threshold = thr_di if (('DI_gender_avg' in avg_col) or ('DI_age_avg' in avg_col) or ('DI_race_avg' in avg_col)) else thr_other  # Set threshold based on DI metrics
                    if references.get(avg_col) is not None and (references[avg_col] - row[avg_col] < threshold):
                        df.at[index, test_col] = 'insignificant-p2'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

    # Save 'false better' rows to a new CSV file
    false_better_df.to_csv(output_false_positives, index=False)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output, index=False)

    return len(false_better_df)



In [ ]:
file_paths = ['../results/test/positive_insig_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_kdd_ttest_results_epochs_intervals.csv']

results_paths = ['../results/test/positive_insig_2_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_kdd_ttest_results_epochs_intervals.csv']

ars = false_positives_fairness(path=file_paths[0], thr_di=1, thr_other=0.01, output_false_positives='../results/test/ars_false_positives_with_regard_to_fairness.csv',output=results_paths[0] )
dc = false_positives_fairness(path=file_paths[1], thr_di=1, thr_other=0.01,output_false_positives='../results/test/dc_false_positives_with_regard_to_fairness.csv', output=results_paths[1])
adult = false_positives_fairness(path=file_paths[2], thr_di=1, thr_other=0.01,output_false_positives='../results/test/adult_false_positives_with_regard_to_fairness.csv', output=results_paths[2])
kdd = false_positives_fairness(path=file_paths[3], thr_di=1, thr_other=0.01,output_false_positives='../results/test/kdd_false_positives_with_regard_to_fairness.csv', output=results_paths[3])
mobiact = false_positives_fairness(path=file_paths[4], thr_di=1, thr_other=0.01,output_false_positives='../results/test/mobiact_false_positives_with_regard_to_fairness.csv', output=results_paths[4])

In [ ]:
import pandas as pd

def concat_filter_csv(file_paths, output):
    # Définition des colonnes communes à extraire de chaque fichier
    common_columns = ['dataset', 'model', 'system', 'ratio', 'test_time', 'test_acc', 
                      'test_f1', 'test_precision', 'test_recall', 'test_SPD_gender', 
                      'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
                      'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
                      'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race']

    # Liste pour stocker les DataFrames
    df_list = []

    # Boucle sur chaque chemin de fichier dans la liste
    for path in file_paths:
        # Lecture du fichier CSV
        df = pd.read_csv(path)
        # Extraction des colonnes communes
        df_common = df[common_columns]
        # Ajout du DataFrame extrait à la liste
        df_list.append(df_common)

    # Concaténation des DataFrames
    result_df = pd.concat(df_list, ignore_index=True)

    # Remplacement de 'significant-p' par 'significant'
    result_df.replace('significant-p2', 'significant', inplace=True)
    result_df.to_csv(output + '-w-significant-p2.csv', index=False)

    # Filtrage des lignes où 'ratio' est '0.5' ou 'system' est 'Random'
    result_df = result_df[(result_df['ratio'] != 0.5) & (result_df['system'] != 'Full')]
    result_df.to_csv(output+'-2c-w-random.csv', index=False)

    result_df = result_df[(result_df['system'] != 'Random')]
    result_df.to_csv(output+'-2C-wo-random.csv', index=False)

    return result_df


In [ ]:
file_paths = ['../results/test/positive_insig_2_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_kdd_ttest_results_epochs_intervals.csv']

output = '../results/test/ttest_5'
concat_filter_csv(file_paths, output)

Utility constraint: the impact is considered significant (positive or negative) if the difference between the full metric and the selection metric exceeds 1%.

In [ ]:
import pandas as pd

def false_positives_utility(path=None,thr_acc=1, thr=0.01, output_false_positives=None, output=None):
    # Load the CSV file
    df = pd.read_csv(path)

    # Initialize DataFrames
    false_better_df = pd.DataFrame()

    # Columns to check for the value 'positive'
    columns_to_modify = [
        'test_acc', 'test_f1', 'test_precision', 'test_recall'
    ]

    # Columns with the average metrics
    columns_to_check = ['accuracy_avg', 'F1_score_avg', 'Precision_avg', 'Recall_avg'
        
    ]

    # Initialize references dictionary
    references = {col: None for col in columns_to_check if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'positive':
                    threshold = thr_acc if (('accuracy_avg' in avg_col)) else thr  # Set threshold based on DI metrics

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-pu'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

    # Save 'false better' rows to a new CSV file
    false_better_df.to_csv(output_false_positives, index=False)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output, index=False)

    return len(false_better_df)



In [ ]:
file_paths = ['../results/test/positive_insig_2_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_2_5_kdd_ttest_results_epochs_intervals.csv']

results_paths =  ['../results/test/positive_insig_3_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_kdd_ttest_results_epochs_intervals.csv']

ars = false_positives_utility(path=file_paths[0], thr_acc=1, thr=0.01, output_false_positives='../results/test/ars_false_positives_utility_with_regard_to_utility.csv',output=results_paths[0] )
dc = false_positives_utility(path=file_paths[1], thr_acc=1, thr=0.01,output_false_positives='../results/test/dc_false_positives_utility_with_regard_to_utility.csv', output=results_paths[1])
adult = false_positives_utility(path=file_paths[2], thr_acc=1, thr=0.01,output_false_positives='../results/test/adult_false_positives_utility_with_regard_to_utility.csv', output=results_paths[2])
kdd = false_positives_utility(path=file_paths[3], thr_acc=1, thr=0.01,output_false_positives='../results/test/kdd_false_positives_utility_with_regard_to_utility.csv', output=results_paths[3])
mobiact = false_positives_utility(path=file_paths[4], thr_acc=1, thr=0.01,output_false_positives='../results/test/mobiact_false_positives_utility_with_regard_to_utility.csv', output=results_paths[4])

Conditions on both positive and negative impact 

In [ ]:
import pandas as pd

def false_positives_fairness(path=None, thr_di=1, thr_other=0.01, output_false_positives=None, output=None):
    # Load the CSV file
    df = pd.read_csv(path)

    # Initialize DataFrames
    false_better_df = pd.DataFrame()

    # Columns to check for the value 'positive'
    columns_to_modify = [
        'test_SPD_gender', 'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
        'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
        'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race'
    ]

    # Columns with the average metrics
    columns_to_check = [
        'SPD_gender_avg', 'EOD_gender_avg', 'AOD_gender_avg', 'DI_gender_avg', 'DcI_gender_avg',
        'SPD_age_avg', 'EOD_age_avg', 'AOD_age_avg', 'DI_age_avg', 'DcI_age_avg',
        'SPD_race_avg', 'EOD_race_avg', 'AOD_race_avg', 'DI_race_avg', 'DcI_race_avg'
    ]

    # Initialize references dictionary
    references = {col: None for col in columns_to_check if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'negative':
                    threshold = thr_di if (('DI_gender_avg' in avg_col) or ('DI_age_avg' in avg_col) or ('DI_race_avg' in avg_col)) else thr_other  # Set threshold based on DI metrics
                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-p3'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

    # Save 'false better' rows to a new CSV file
    false_better_df.to_csv(output_false_positives, index=False)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output, index=False)

    return len(false_better_df)



In [ ]:
file_paths =  ['../results/test/positive_insig_3_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_3_5_kdd_ttest_results_epochs_intervals.csv']

results_paths =  ['../results/test/positive_insig_4_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_kdd_ttest_results_epochs_intervals.csv']

ars = false_positives_fairness(path=file_paths[0], thr_di=1, thr_other=0.01, output_false_positives='../results/test/ars_false_positives_with_regard_to_negative_fairness.csv',output=results_paths[0] )
dc = false_positives_fairness(path=file_paths[1], thr_di=1, thr_other=0.01,output_false_positives='../results/test/dc_false_positives_with_regard_to_negative_fairnessn.csv', output=results_paths[1])
adult = false_positives_fairness(path=file_paths[2], thr_di=1, thr_other=0.01,output_false_positives='../results/test/adult_false_positives_with_regard_to_negative_fairness.csv', output=results_paths[2])
kdd = false_positives_fairness(path=file_paths[3], thr_di=1, thr_other=0.01,output_false_positives='../results/test/kdd_false_positives_with_regard_to_negative_fairness.csv', output=results_paths[3])
mobiact = false_positives_fairness(path=file_paths[4], thr_di=1, thr_other=0.01,output_false_positives='../results/test/mobiact_false_positives_with_regard_to_negative_fairness.csv', output=results_paths[4])

In [ ]:
import pandas as pd

def false_positives_utility(path=None,thr_acc=1, thr=0.01, output_false_positives=None, output=None):
    # Load the CSV file
    df = pd.read_csv(path)

    # Initialize DataFrames
    false_better_df = pd.DataFrame()

    # Columns to check for the value 'positive'
    columns_to_modify = [
        'test_acc', 'test_f1', 'test_precision', 'test_recall'
    ]

    # Columns with the average metrics
    columns_to_check = ['accuracy_avg', 'F1_score_avg', 'Precision_avg', 'Recall_avg'
        
    ]

    # Initialize references dictionary
    references = {col: None for col in columns_to_check if col in df.columns}

    # Iterate through DataFrame
    for index, row in df.iterrows():
        if row['system'] == 'Full':
            references = {col: None for col in columns_to_check if col in df.columns}
            # Set the reference values for metrics when system is 'Full'
            for col in columns_to_check:
                if col in df.columns:
                    references[col] = row[col]
        else:
            # Check other rows where system is not 'Full'
            for test_col, avg_col in zip(columns_to_modify, columns_to_check):
                if test_col in df.columns and avg_col in df.columns and row[test_col] == 'negative':
                    threshold = thr_acc if (('accuracy_avg' in avg_col)) else thr  # Set threshold based on DI metrics

                    if references.get(avg_col) is not None and (abs(references[avg_col] - row[avg_col]) < threshold):
                        df.at[index, test_col] = 'insignificant-pu2'
                        if row['system'] != 'Random' and row['ratio'] != 0.5:
                            # Add row to 'false better' DataFrame if certain conditions are met
                            false_better_df = pd.concat([false_better_df, pd.DataFrame([row])], ignore_index=True)

    # Save 'false better' rows to a new CSV file
    false_better_df.to_csv(output_false_positives, index=False)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output, index=False)

    return len(false_better_df)



In [ ]:
file_paths = ['../results/test/positive_insig_4_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_4_5_kdd_ttest_results_epochs_intervals.csv']

results_paths =  ['../results/test/positive_insig_5_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_kdd_ttest_results_epochs_intervals.csv']

ars = false_positives_utility(path=file_paths[0], thr_acc=1, thr=0.01, output_false_positives='../results/test/ars_false_negatives_utility_with_regard_to_utility.csv',output=results_paths[0] )
dc = false_positives_utility(path=file_paths[1], thr_acc=1, thr=0.01,output_false_positives='../results/test/dc_false_negatives_utility_with_regard_to_utility.csv', output=results_paths[1])
adult = false_positives_utility(path=file_paths[2], thr_acc=1, thr=0.01,output_false_positives='../results/test/adult_false_negatives_utility_with_regard_to_utility.csv', output=results_paths[2])
kdd = false_positives_utility(path=file_paths[3], thr_acc=1, thr=0.01,output_false_positives='../results/test/kdd_false_negatives_utility_with_regard_to_utility.csv', output=results_paths[3])
mobiact = false_positives_utility(path=file_paths[4], thr_acc=1, thr=0.01,output_false_positives='../results/test/mobiact_false_negatives_utility_with_regard_to_utility.csv', output=results_paths[4])

Concat outputs 

In [ ]:
import pandas as pd

def concat_filter_csv(file_paths, output):
    # Définition des colonnes communes à extraire de chaque fichier
    common_columns = ['dataset', 'model', 'system', 'ratio', 'test_time', 'test_acc', 
                      'test_f1', 'test_precision', 'test_recall', 'test_SPD_gender', 
                      'test_EOD_gender', 'test_AOD_gender', 'test_DI_gender', 'test_DcI_gender', 
                      'test_SPD_age', 'test_EOD_age', 'test_AOD_age', 'test_DI_age', 'test_DcI_age', 
                      'test_SPD_race', 'test_EOD_race', 'test_AOD_race', 'test_DI_race', 'test_DcI_race']

    # Liste pour stocker les DataFrames
    df_list = []

    # Boucle sur chaque chemin de fichier dans la liste
    for path in file_paths:
        # Lecture du fichier CSV
        df = pd.read_csv(path)
        # Extraction des colonnes communes
        df_common = df[common_columns]
        # Ajout du DataFrame extrait à la liste
        df_list.append(df_common)

    # Concaténation des DataFrames
    result_df = pd.concat(df_list, ignore_index=True)

    # Remplacement de 'significant-p' par 'significant'
    result_df.to_csv(output + '-w-significant-p-5C.csv', index=False)

    result_df.replace('insignificant-p', 'insignificant', inplace=True)

    result_df.replace('insignificant-p2', 'insignificant', inplace=True)

    result_df.replace('insignificant-p3', 'insignificant', inplace=True)

    result_df.replace('insignificant-pu', 'insignificant', inplace=True)

    result_df.replace('insignificant-pu2', 'insignificant', inplace=True)

    result_df.to_csv(output + '-wo-significant-p-5C.csv', index=False)

    # Filtrage des lignes où 'ratio' est '0.5' ou 'system' est 'Random'
    result_df = result_df[(result_df['ratio'] != 0.5) & (result_df['system'] != 'Full')]
    result_df.to_csv(output+'-5c-w-random.csv', index=False)

    result_df = result_df[(result_df['system'] != 'Random')]
    result_df.to_csv(output+'-5C-wo-random.csv', index=False)

    return result_df


In [ ]:
file_paths =  ['../results/test/positive_insig_5_5_ars_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_dc_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_mobiact_ttest_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_adult_ttest_results_epochs_intervals.csv', 
              '../results/test/positive_insig_5_5_kdd_ttest_results_epochs_intervals.csv']

output = '../results/test/ttest_5'
concat_filter_csv(file_paths, output)